# Propagate structure

In [1]:
from utils.testproblems import generate_random_prob
from utils.polycasebuilder import generate_random_polynomials
from graph.graphutils import edges_to_Ein_Eout, sources, flat_graph_formulation
from graph.operators import sort_scc
from torchengine import AnalyticalSetSympy, AnalysisFunction, EliminateAnalysis, ParallelAnalysis, Function
import torch

In [2]:
indices = {
    'x1': torch.tensor([0]), 
    'x2': torch.tensor([1]), 
    'x3': torch.tensor([2]),
    'x4': torch.tensor([3,4]),
    'x5': torch.tensor([5,6]),
    'x6': torch.tensor([7])
    }

A1 = AnalysisFunction((('x1','x2','x3'), ('x5', 'x4'), lambda x1,x2,x3: (torch.tensor((x1,x2)), 
                                                                         torch.tensor((x3, x1*x2*x3)))), indices)
A2 = AnalysisFunction((('x3','x1'), ('x6',), lambda x3,x1: x3+x1), indices)
A3 = AnalysisFunction((('x6',), ('x1',), lambda x6: x6), indices)
F = Function((('x6',), lambda x6: x6), indices)
A123 = EliminateAnalysis([A1, A2, A3], [])

In [3]:
F.structure_full

([7],)

In [4]:
residualvar = [0]
A123res = ParallelAnalysis([A123], [F], sharedvars=residualvar)

In [5]:
A123res.structure

(tensor([0, 1, 2]), tensor([]))

In [6]:
x0 = torch.tensor([1,2,3,4,5,6,7,8], dtype=torch.float64)
A3(x0)

tensor([8., 2., 3., 4., 5., 6., 7., 8.], dtype=torch.float64)

In [7]:
n_eqs, n_vars, sparsity, seed1, seed2 = 10, 15, 1.5, 42, 12345 #1.7
eqv, varinc, dout = generate_random_prob(n_eqs, n_vars, seed1, sparsity)
polynomials, var_mapping, edges, tree = generate_random_polynomials(eqv, dout, n_eqs, seed=seed2)
symb_mapping = {key: elt[0] for key, elt in var_mapping.items()}
inputids = sorted(sources(*edges_to_Ein_Eout(edges)))
inputvars = [var_mapping[elt][0] for elt in inputids]
fobj = sum([(elt-1)**2 for elt in inputvars])
indices = {elt: torch.tensor([int(i)]) for i, elt in symb_mapping.items()}

In [8]:
sets = {idx:AnalyticalSetSympy(poly, indices=indices).reassign(symb_mapping[edges[1][idx][0]]) for idx,poly in polynomials.items()}

In [9]:
sets[0].analysis.structure, sets[0].analysis.structure_full

((tensor([12,  3,  7,  6]), tensor([4])), {4: [12, 3, 7, 6]})

In [10]:
import torch
import itertools

analysis_structures = (
    {5: [1,2,3], 4: [1,2,3]},
    {5: [3,4]},
    {1: [5]}
    )
functional_structure = (
    [4],
    [5], 
    [6],
)

eliminated_output = {}
full_structure = {}
for structure in analysis_structures:
    eliminated_output_buffer = {}
    for struct_out, struct_in in structure.items():
        full_structure[struct_out] = []
        for i in struct_in:
            extension = [i] if i not in eliminated_output else eliminated_output[i]
            for elt in extension:
                if elt not in full_structure[struct_out]:
                    full_structure[struct_out] += [elt] # the last part is to avoid self reference
        eliminated_output_buffer[struct_out] = eliminated_output.get(struct_out,[])+full_structure[struct_out]
    eliminated_output.update(eliminated_output_buffer)

full_structure

{5: [3, 1, 2], 4: [1, 2, 3], 1: [1, 2, 3]}

In [11]:
get_analysis_structure(analysis_structures)

NameError: name 'get_analysis_structure' is not defined

In [1]:
from torchengine import AnalyticalSet, Function, EliminateAnalysis, EliminateAnalysisMergeResiduals, ElimResidual
from torchengine import ParallelAnalysis
from scipy.optimize import minimize
import numpy as np
import torch
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

In [2]:
from trash.inputresolver import reassigneq
import sympy as sp
# need a AnalyticalSetSympa that has a transformation function to generate an alternate AnaltyicalSetSympa with a different output set using reassingeq
class AnalyticalSetSympy(AnalyticalSet):
    def __init__(self, expression, outputvar=None, indices=None):
        outputvars = (outputvar,) if outputvar is not None else ()
        outputvar = 0 if outputvar is None else outputvar
        self.indices = indices
        self.expression = expression
        self.outputvar = outputvar
        self.variables = tuple(expression.free_symbols)
        residual_variables =self.variables+outputvars
        analysis_function = sp.lambdify(self.variables, expression, torch)
        residual_function = sp.lambdify(residual_variables, expression-outputvar, torch)
        triplet = (self.variables, outputvars, analysis_function)
        tuplet = (residual_variables, residual_function)
        super().__init__(triplet, indices, forceresidual=tuplet)

    def reassign(self, new_outputvar):
        outputvar = 0
        if self.outputvar != 0:
            outputvar = self.outputvar
        newexpr = reassigneq(None, self.expression-outputvar, new_outputvar)
        return AnalyticalSetSympy(newexpr, new_outputvar, self.indices)

# Example

In [4]:
n_eqs, n_vars, sparsity, seed1, seed2 = 10, 15, 1.5, 42, 12345 #1.7
eqv, varinc, dout = generate_random_prob(n_eqs, n_vars, seed1, sparsity)
polynomials, var_mapping, edges, tree = generate_random_polynomials(eqv, dout, n_eqs, seed=seed2)
symb_mapping = {key: elt[0] for key, elt in var_mapping.items()}
inputids = sorted(sources(*edges_to_Ein_Eout(edges)))
inputvars = [var_mapping[elt][0] for elt in inputids]
fobj = sum([(elt-1)**2 for elt in inputvars])
G = flat_graph_formulation(*edges)
indices = {elt: torch.tensor([int(i)]) for i, elt in symb_mapping.items()}
sets = {idx:AnalyticalSetSympy(poly, indices=indices).reassign(symb_mapping[edges[1][idx][0]]) for idx,poly in polynomials.items()}

In [5]:
indices = {elt: torch.tensor([int(i)]) for i, elt in symb_mapping.items()}

In [6]:
sets = {idx:AnalyticalSetSympy(poly, indices=indices).reassign(symb_mapping[edges[1][idx][0]]) for idx,poly in polynomials.items()}

In [7]:
order = sort_scc(G)
eqconstraints = []
elimination = []
edges_out = edges[1]
for elt in order:
    if len(elt) > 1: # solve group
        groupsets = [sets[eq.name] for eq in elt]
        residuals = [s.residual for s in groupsets]
        coupled = EliminateAnalysisMergeResiduals([], residuals)
        coupledvars = [s.outputvar for s in groupsets]
        solver = ElimResidual(coupled, coupledvars, indices) #indices
        elimination.append(solver)
        # alterantively we could add them to elimination
    else:
        eqid = elt.pop().name
        aset = sets[eqid]
        if edges_out[eqid] == ():
            eqconstraints.append(aset.residual)
        else:
            elimination.append(aset.analysis) 

In [8]:
torch.manual_seed(43)
x0 = torch.rand(n_vars, dtype=torch.float64)

In [16]:
xsol = elimination[1](x0)

# Testing Sympy standalone

In [49]:
# sellar2 = (('x1', 'x2', 'u1'), ('u2',), lambda x1, x2, u1: u1**0.5 + x1 + x2)
# use AnalyticalSetSympy instead:
x1, x2, u1, u2 = sp.symbols('x1 x2 u1 u2')
varorder = [x1, x2, u1, u2]
indices = {elt: torch.tensor([i]) for i, elt in enumerate(varorder)}
sellar2 = AnalyticalSetSympy(u1**0.5 + x1 + x2, u2, indices)
x0 = torch.tensor([1,1,9,5], dtype=torch.float64)
sellar2.analysis(x0).numpy()

array([1, 1, 9, 5])

In [50]:
sellar2.reassign(x2).analysis(x0).numpy()

array([1, 1, 9, 5])

In [ ]:
#order = [{1}, {2,3,4}, {5,6}]
#eliminate = [{1}, {2,3,4}], eqcons = [{5,6}] <- residuals are used for these
# in eliminate any single (e.g. {1}) component will be used to generate an analysis based on reassign eq
order = [{0}, {1,2,3}, {4,5}, {6}]
edges_in = {
    0: ('x','u1','u2'),
    1: ('x','u1','u2'),
    2: ('x','u1','u2'),
    3: ('x','u1','u2'),
    4: ('x','u1','u2'),
    5: ('x','u1','u2'),
    6: ('x','u1','u2')
}
edges_out = {
    0: ('u2',),
    1: ('u2',),
    2: ('u2',),
    3: ('u2',),
    4: ('u2',),
    5: ('u2',),
    6: ()
}
eqconstraints = []
elimination = []
for elt in order:
    if len(elt) > 1: # solve group
        residuals = [s.residual for s in elt]
        coupled = EliminateAnalysisMergeResiduals([], residuals)
        coupledvars = [s.outputvar for s in elt]
        solver = ElimResidual(coupled, coupledvars, indices) #indices
        elimination.append(solver)
    elif edges_out[elt[0]] == ():
        eqconstraints.append(elt[0].residual)
    else:
        elimination.append(elt[0].analysis)     

In [2]:
sellarobj = (('x2', 'x3', 'u1', 'u2'), lambda x2, x3, u1, u2: x3**2 + x2 + u1 + np.exp(-u2))
sellar1a = (('x1', 'x2'), ('u3',), lambda x1, x2: x1**2 + x2)
sellar1b = (('x3', 'u2', 'u3'), ('u1',), lambda x3, u2, u3: u3 + x3 - 0.2*u2)
sellar2 = (('x1', 'x2', 'u1'), ('u2',), lambda x1, x2, u1: u1**0.5 + x1 + x2)
ineqcon1 = (('u1',), lambda u1: 1-u1/3.16)
ineqcon2 = (('u2',), lambda u2: u2/24-1)

varorder = ['x1','x2','x3','u1','u2','u3']
indices = {elt: torch.tensor([i]) for i, elt in enumerate(varorder)}

In [3]:
edges = (
    {
        'sellar1a': ('x1', 'x2'),
        'sellar1b': ('x3', 'u2', 'u3'),
        'sellar2': ('x1', 'x2', 'u1'),
        'sellarobj': ('x2', 'x3', 'u1', 'u2'),
        'ineqcon1': ('u1',),
        'ineqcon2': ('u2',)
    },
    {
        'sellar1a': ('u3',),
        'sellar1b': ('u1',),
        'sellar2': ('u2',),
        'sellarobj': (),
        'ineqcon1': (),
        'ineqcon2': ()
    }
)

In [ ]:
#order = [{1}, {2,3,4}, {5,6}] # filter out end comps to get
#eliminate = [{1}, {2,3,4}], eqcons = [{5,6}] <- residuals are used for these
# in eliminate any single (e.g. {1}) component will be used to generate an analysis based on reassign eq
# when there are multiple components we use an ElimResidual

In [4]:
set1a = AnalyticalSet(sellar1a, indices)
set1b = AnalyticalSet(sellar1b, indices)
set2 = AnalyticalSet(sellar2, indices)

coupled = EliminateAnalysisMergeResiduals([],[set1a.residual, set1b.residual, set2.residual])
coupledvars = ['u1','u2','u3']
solver = ElimResidual(coupled, coupledvars, indices)

con1 = Function(ineqcon1, indices)
con2 = Function(ineqcon2, indices)
ineqcons = EliminateAnalysisMergeResiduals([], [con1, con2])
obj = Function(sellarobj, indices)

In [18]:
empty_eq_cons = Function((tuple(), lambda : torch.tensor([])), indices)

## Architectures

In [19]:
MDF = EliminateAnalysis([solver], [obj, ineqcons, empty_eq_cons])

In [7]:
set1 = EliminateAnalysis([set1a.analysis, set1b.analysis], [])

In [8]:
set1.structure,  set2.analysis.structure

((tensor([0, 1, 2, 4]), tensor([5, 3])), (tensor([0, 1, 3]), tensor([4])))

In [9]:
IDF = ParallelAnalysis([set1, set2.analysis], 
                       [obj, ineqcons], sharedvars=['u1','u2'], indices=indices)

In [10]:
IDF.structure

(tensor([0, 1, 2, 4, 3]), ())

In [11]:
AAO = EliminateAnalysis([], [obj, ineqcons, coupled])

### Run it

In [12]:
x0 = torch.tensor([0.5,.5,0.5,0.1,0.1,0.1], dtype=torch.float64)

In [13]:
AAO(x0)

[tensor([1.7548], dtype=torch.float64),
 tensor([ 0.9684, -0.9958], dtype=torch.float64),
 tensor([-0.6500, -0.4800, -1.2162], dtype=torch.float64)]

In [14]:
IDF(x0)

[tensor([2.2481], dtype=torch.float64),
 tensor([ 0.6108, -0.9452], dtype=torch.float64),
 tensor([1.1300, 1.2162], dtype=torch.float64)]

In [20]:
MDF(x0)

[tensor([1.7593], dtype=torch.float64),
 tensor([ 0.7266, -0.9196], dtype=torch.float64),
 tensor([])]

In [21]:
x0

tensor([0.5000, 0.5000, 0.5000, 0.8641, 1.9296, 0.7500], dtype=torch.float64)